In [1]:
# Cloning shrinkai
!git clone https://github.com/nikshrimali/shrinkai

fatal: destination path 'shrinkai' already exists and is not an empty directory.


In [2]:
# Importing Libraries
import torch
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau


from shrinkai.model.model_test import model_testing
from shrinkai.model.model_train import model_training

from shrinkai.data_process.getdata import GetCIFAR10_TrainData
from shrinkai.data_process.misclassified_data import *

import matplotlib.pyplot as plt
%matplotlib inline

device = "cuda" if torch.cuda.is_available else "cpu"
print(device)

Compose(
    RandomHorizontalFlip(p=0.5)
    RandomRotation(degrees=(-10, 10), resample=False, expand=False)
    RandomAffine(degrees=(0, 0), scale=(0.8, 1.2), shear=(-10, 10))
    ColorJitter(brightness=[0.8, 1.2], contrast=[0.8, 1.2], saturation=[0.8, 1.2], hue=None)
    ToTensor()
    Normalize(mean=(0.491, 0.482, 0.446), std=(0.247, 0.243, 0.261))
)
cuda


In [3]:
# Importing Model and printing Summary
from shrinkai.model.resnetmodel8 import ResNet18
model = ResNet18().to(device)
# model = F.log_softmax(model, dim=-1).to(device)
summary(model, input_size=(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
           Dropout-5           [-1, 64, 32, 32]               0
            Conv2d-6           [-1, 64, 32, 32]          36,864
       BatchNorm2d-7           [-1, 64, 32, 32]             128
           Dropout-8           [-1, 64, 32, 32]               0
        BasicBlock-9           [-1, 64, 32, 32]               0
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
          Dropout-12           [-1, 64, 32, 32]               0
           Conv2d-13           [-1, 64, 32, 32]          36,864
      BatchNorm2d-14           [-1, 64,

In [4]:
# Downloading the dataset and applying transformations
import os
print(os.getcwd())
dir = os.chdir('D:\Python Projects\EVA')
print(os.getcwd())
data = GetCIFAR10_TrainData(dir)
trainset = data.download_train_data()
testset = data.download_test_data()
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512,
                                          shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=512,
                                         shuffle=False, num_workers=4)

d:\Python Projects\EVA\Assignment-8
D:\Python Projects\EVA
Files already downloaded and verified
Files already downloaded and verified


In [5]:
# Training the model

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=5, threshold=0.001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=True)
# scheduler = StepLR(optimizer, step_size=15, gamma=0.1)

train_acc = []
train_losses = []
test_acc = []
test_losses = []

EPOCHS = 15

for i in range(EPOCHS):
    print(f'EPOCHS : {i}')
    model_training(model, device, trainloader, optimizer, train_acc, train_losses, l1_loss=False)
    scheduler.step(train_losses[-1])
    misclassified = model_testing(model, device, testloader, test_acc, test_losses)

  0%|          | 0/98 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [6]:

fig, axs = plt.subplots(2,2, figsize=(25,20))

axs[0,0].set_title('Train Losses')
axs[0,1].set_title('Training Accuracy')
axs[1,0].set_title('Test Losses')
axs[1,1].set_title('Test Accuracy')

axs[0,0].plot(train_losses)
axs[0,1].plot(train_acc)
axs[1,0].plot(test_losses)
axs[1,1].plot(test_acc)

# axs[0,0].legend(list_modes)
# axs[0,1].legend(list_modes)
# axs[1,0].legend(list_modes)
# axs[1,1].legend(list_modes)
# fig.savefig(os.path.join(img_dir, 'metrics.png'))

NameError: name 'plt' is not defined